In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


import sys 
import os
path = os.path.dirname(os.getcwd())
sys.path.append(path)
from lib import *

# Data Import
df = pd.read_csv(r'../data/DE_2021.csv')
ci_data_2021 = df['carbon_intensity_avg']

df = pd.read_csv(r'../data/projectcount_wikiDE_2015.csv')
clickData_hourly_2015 = df["De"]

costantCB = pd.read_csv(r'../data/CB_constant.csv')
adaptiveCB = pd.read_csv(r'../data/CB_adaptive.csv')

totalCB = 0

In [ ]:
for inputVersion in range(4):

    if inputVersion == 0:
        y = "HH"
    elif inputVersion == 1:
        y = "HL"
    elif inputVersion == 2:
        y = "LH"
    elif inputVersion == 3:
        y = "LL"
    
    for CB_version in range(8):

        if CB_version == 0:
            x = "CONSTANT_MAX_CB_"
            carbonBudget = costantCB["MaxCarbonBudget"].values[0]
        elif CB_version == 1:
            x = "CONSTANT_HIGH_CB_"
            carbonBudget = costantCB["HighCarbonBudget"].values[0]
        elif CB_version == 2:
            x = "CONSTANT_AVG_CB_"
            carbonBudget = costantCB["AVGCarbonBudget"].values[0]
        elif CB_version == 3:
            x = "CONSTANT_LOW_CB_"
            carbonBudget = costantCB["LowCarbonBudget"].values[0]
        elif CB_version == 4:
            x = "CONSTANT_MIN_CB_"
            carbonBudget = costantCB["MinCarbonBudget"].values[0]
        elif CB_version == 5:
            x = "ADAPTIVE_MAX_CB_"
            carbonBudget = adaptiveCB["MaxCarbonBudget"]
        elif CB_version == 6:
            x = "ADAPTIVE_AVG_CB_"
            carbonBudget = adaptiveCB["AVGCarbonBudget"]
        elif CB_version == 7:
            x = "ADAPTIVE_MIN_CB_"  
            carbonBudget = adaptiveCB["MinCarbonBudget"]


        # Results Import
        optimization_result = pd.read_csv("../results/" + y + "/optimizedCarbonAware/" + x + "optimization_result_" + y + ".csv")
        highPerformance_result = pd.read_csv("../results/" + y + "/highPerformanceBaseline_" + y + ".csv")
        simpleCarbonAware_result = pd.read_csv("../results/" + y + "/simpleCarbonAware/" + x + "simpleCarbonAwareBaseline_" + y + ".csv")
        sequentialCarbonAware_result = pd.read_csv("../results/" + y + "/sequentialCarbonAware/" + x + "sequentialCarbonAwareBaseline_" + y + ".csv")

        totalCB = 0
        maxRevenue = 0
        opt_totalEmissions = 0
        hp_totalEmissions = 0
        sim_totalEmissions = 0
        seq_totalEmissions = 0
        opt_totalRevenue = 0
        hp_totalRevenue = 0
        sim_totalRevenue = 0
        seq_totalRevenue = 0
        opt_overBudget = 0
        hp_overBudget = 0
        sim_overBudget = 0
        seq_overBudget = 0
        opt_QoE = 0
        hp_QoE = 0
        sim_QoE = 0
        seq_QoE = 0

        for i in range(len(ci_data_2021)):
            if CB_version <= 4: 
                totalCB += carbonBudget
                cb = carbonBudget
            else:
                totalCB += carbonBudget[i]
                cb = carbonBudget[i]
            # Calcolo emissioni totali
            opt_totalEmissions += optimization_result["ce"][i]
            hp_totalEmissions += highPerformance_result["Energy demand"][i] * ci_data_2021[i]
            sim_totalEmissions += simpleCarbonAware_result["ce"][i]
            seq_totalEmissions += sequentialCarbonAware_result["ce"][i]

            # Calcolo numero di over budget orari
            opt_overBudget += (optimization_result["ce"][i] > cb)
            hp_overBudget += ((highPerformance_result["Energy demand"][i] * ci_data_2021[i]) > cb)
            sim_overBudget += (simpleCarbonAware_result["ce"][i] > cb)
            seq_overBudget += (sequentialCarbonAware_result["ce"][i] > cb)

            # Calcolo QoE totali
            opt_QoE += optimization_result["QoE"][i]
            hp_QoE += 1
            sim_QoE += simpleCarbonAware_result["QoE"][i]
            seq_QoE += sequentialCarbonAware_result["QoE"][i]


            # Calcolo revenue totali
            opt_totalRevenue += optimization_result["total rev"][i]
            hp_totalRevenue += highPerformance_result["Revenue"][i]
            sim_totalRevenue += simpleCarbonAware_result["Revenue"][i]
            seq_totalRevenue += sequentialCarbonAware_result["Revenue"][i]
            
        maxRevenue = hp_totalRevenue

        print(x, " with input ", y)
        print("Total CB: ", totalCB)
        print("- Optimizer: ")
        print("Total emissions: ", opt_totalEmissions, " -> ", '%.3f'%(opt_totalEmissions*100/totalCB), "%")
        print("Number of times overbudget: ", opt_overBudget, " -> ", '%.3f'%(opt_overBudget*100/len(ci_data_2021)), "%")
        print("Total revenue: ", opt_totalRevenue, " -> ", '%.3f'%(opt_totalRevenue*100/maxRevenue), "%")
        print("QoE: ", '%.3f'%(opt_QoE / len(ci_data_2021)))
        print("- High performance baseline: ")
        print("Total emissions: ", hp_totalEmissions, '%.3f'%(hp_totalEmissions*100/totalCB), "%")
        print("Number of times overbudget: ", hp_overBudget, " -> ", '%.3f'%(hp_overBudget*100/len(ci_data_2021)), "%")
        print("Total revenue: ", hp_totalRevenue, " -> ", '%.3f'%(hp_totalRevenue*100/maxRevenue), "%")  
        print("QoE: ", '%.3f'%(hp_QoE / len(ci_data_2021))) 
        print("- Simple carbon aware: ")
        print("Total emissions: ", sim_totalEmissions, '%.3f'%(sim_totalEmissions*100/totalCB), "%")
        print("Number of times overbudget: ", sim_overBudget, " -> ", '%.3f'%(sim_overBudget*100/len(ci_data_2021)), "%")
        print("Total revenue: ", sim_totalRevenue, " -> ", '%.3f'%(sim_totalRevenue*100/maxRevenue), "%")  
        print("QoE: ", '%.3f'%(sim_QoE / len(ci_data_2021)))     
        print("- Sequential carbon aware: ")
        print("Total emissions: ", seq_totalEmissions, '%.3f'%(seq_totalEmissions*100/totalCB), "%")
        print("Number of times overbudget: ", seq_overBudget, " -> ", '%.3f'%(seq_overBudget*100/len(ci_data_2021)), "%")
        print("Total revenue: ", seq_totalRevenue, " -> ", '%.3f'%(seq_totalRevenue*100/maxRevenue), "%")     
        print("QoE: ", '%.3f'%(seq_QoE / len(ci_data_2021))) 
        print("-------------------------------------------------------")


CONSTANT_MAX_CB_  with input  HH
Total CB:  653326357789.6827
- Optimizer: 
Total emissions:  95805585329.8882  ->  14.664 %
Number of times overbudget:  0  ->  0.000 %
Total revenue:  16815513104.542253  ->  99.983 %
QoE:  1.000
- High performance baseline: 
Total emissions:  95858146038.1972 14.672 %
Number of times overbudget:  1  ->  0.011 %
Total revenue:  16818424811.024452  ->  100.000 %
QoE:  1.000
- Simple carbon aware: 
Total emissions:  95775408318.46721 14.660 %
Number of times overbudget:  0  ->  0.000 %
Total revenue:  16806301562.194551  ->  99.928 %
QoE:  1.000
- Sequential carbon aware: 
Total emissions:  95775408318.46721 14.660 %
Number of times overbudget:  0  ->  0.000 %
Total revenue:  16806301562.194551  ->  99.928 %
QoE:  1.000
-------------------------------------------------------
CONSTANT_HIGH_CB_  with input  HH
Total CB:  497038477566.59796
- Optimizer: 
Total emissions:  95773075580.2342  ->  19.269 %
Number of times overbudget:  0  ->  0.000 %
Total reven